<a href="https://colab.research.google.com/github/mlluizza/AIRBNB-RIO-DE-JANEIRO/blob/main/AIRBNB_RIO_DE_JANEIRO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto Airbnb Rio - Ferramenta de Previsão de Preço de Imóvel para pessoas comuns**

# **Contexto**

No Airbnb, qualquer pessoa que tenha um quarto um imóvel de qualquer tipo (apartamento, casa, chalé, pousada, etc.) pode ofertar o seu imóvel para ser alugado por diária.

Você cria o seu perfil de host (pessoa que disponibiliza um imóvel para aluguel por diária) e cria o anúncio do seu imóvel.

Nesse anúncio, o host deve descrever as características do imóvel da forma mais completa possível, de forma a ajudar os locadores/viajantes a escolherem o melhor imóvel para eles(e de forma a tornar o seu anúncio mais atrativo)

Existem dezenas de personalizações possíveis no seu anúncio,desde quantidade mínima de diária, preço, quantidade de quartos, até regras de cancelamento, taxa extra para hóspedes exras, exigências de verificaçã de identidade do locador, etc.


# **Objetivo**

Construir um modelo de previsão de preço que permita um pessoa comum que possui um imóvel possa sqaber quanto deve cobrar pela diária do seu imóvel.

Ou ainda, para o locador comum, dado o imóvel que ele está buscando, ajudar a saber se aquele imóvel está com preço atrativo (abaixo da média para imóveis com as mesma características) ou não.